In [1]:
import numpy as np
import copy
import networkx as nx

class Position:
    __instances = {}
    def __new__(cls, *params):
        if (params in Position.__instances):
            return Position.__instances[params]

        obj = super().__new__(Position)
        obj.__init__(*params)

        Position.__instances[params] = obj

        return obj

    def __init__(self, i: int, j: int):
        self.i = i
        self.j = j

    def __eq__(self, ij):
        pos = Position(*ij)
        return (self.i == pos.i and self.j == pos.j)

    def __iter__(self):
        yield from (self.i, self.j)

    def __repr__(self):
        return str(tuple([*self]))

    def __hash__(self):
        return int(str(self.i) + str(self.j))

In [2]:
test_cases = [
    [
        a := [50, 50, 100],
        b := [40, 90, 70],
        c := [
                [2, 5, 3],
                [4, 3, 2],
                [5, 1, 2],
        ],
        res := [
            [40, 0, 10],
            [0, 0, 50],
            [0, 90, 10],
        ],
    ],
    [
        a := [90, 90, 100],
        b := [50, 100, 130],
        c := [
                [2, 5, 4],
                [7, 6, 5],
                [9, 8, 10],
        ],
        res := [
            [50, 0, 40],
            [0, 0, 90],
            [0, 100, 0],
        ],
    ],
    [
        a := [140, 80, 80],
        b := [100, 100, 100],
        c := [
                [1, 1, 4],
                [5, 2, 1],
                [6, 1, 3],
        ],
        res := [
            [100, 40, 0],
            [0, 0, 80],
            [0, 60, 20],
        ],
    ],
]

In [3]:
def matrix_transport_problem(a, b, c):
    a = a[:]
    b = b[:]
    c = copy.deepcopy(c)

    # балансируем задачу
    sum_a = sum(a)
    sum_b = sum(b)

    if sum_a < sum_b:
        a += [sum_b - sum_a]
        c += [[0] * len(b)]
    elif sum_a > sum_b:
        b += [sum_a - sum_b]
        c = [ci + [0] for ci in c]

    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    n, m = c.shape

    # первая фаза метода потенциалов
    # находим начальный базисный план перевозок
    # с соответствующим множеством базисных позиций,
    # с помощью метода "северо-западного угла"
    x = np.zeros((m, n))
    B = []

    a_copy = np.copy(a)
    b_copy = np.copy(b)

    i, j = 0, 0
    while i < m:
        while j < n:
            minimal = min(a_copy[i], b_copy[j])
            
            x[i][j] = minimal
            a_copy[i] -= minimal
            b_copy[j] -= minimal

            B.append(Position(i, j))

            if a_copy[i] == 0:
                i += 1
                break

            j += 1

    # вторая фаза метода потенциалов
    while True:
        syst = [[1] + [0] * (m + n - 1)]
        b = [0]

        u_offset = 0
        v_offset = m

        for pos in B:
            syst.append([0.] * (m + n))

            syst[-1][u_offset + pos.i] = 1
            syst[-1][v_offset + pos.j] = 1

            b.append(c[pos.i][pos.j])
        
        res = np.linalg.solve(syst, b)

        u = res[:v_offset]
        v = res[v_offset:]

        # проверяем условие оптимальности
        # текущего базисного плана
        for i in range(len(u)):
            for j in range(len(v)):
                if ((i, j) not in B) and (u[i] + v[j] > c[i][j]):                  
                    B.append(Position(i, j))
                    break
            else:
                continue

            break
        else:
            print("Базисный план перевозок с минимальной стоимостью:")
            print(x)
            return

        new_pos = B[-1]
        
        # находим цикл
        # создаем список смежности
        nodes = []
        edges = []

        for node in B:
            nodes.append((node.i, node.j))

            for i in range(node.i - 1, -1, -1):
                if (i, node.j) in B:
                    edges.append(((i, node.j), (node.i, node.j)))
                    break
            for i in range(node.i + 1, n):
                if (i, node.j) in B:
                    edges.append(((i, node.j), (node.i, node.j)))
                    break

            for j in range(node.j - 1, -1, -1):
                if (node.i, j) in B:
                    edges.append(((node.i, j), (node.i, node.j)))
                    break
            for j in range(node.j + 1, m):
                if (node.i, j) in B:
                    edges.append(((node.i, j), (node.i, node.j)))
                    break

        # находим циклы
        G = nx.MultiDiGraph()
        G.add_nodes_from(nodes)
        G.add_edges_from(edges)

        cycle: list
        for found_cycle in list(nx.simple_cycles(G)):
            if new_pos in found_cycle and len(found_cycle) > 2:
                cycle = [Position(i,j) for (i,j) in found_cycle]
                break
        
        while (cycle[0] != new_pos):
            cycle.append(cycle[0])
            cycle.remove(cycle[0])

        # отмечаем угловые вершины
        ninety_degrees = []
        for i in range(len(cycle)):
            i_pre = i - 1
            i_next = (i + 1) % len(cycle)

            pos_pre = cycle[i_pre]
            pos_next = cycle[i_next]
            pos = cycle[i]

            if (pos_pre.i == pos.i and pos_next.j == pos.j or \
                pos_pre.j == pos.j and pos_next.i == pos.i):
                ninety_degrees.append(pos)
            

        # помечаем угловые вершины '+' или '-'      
        pluses = ninety_degrees[::2]

        # изменяем кол-во продукции на угловых позициях,
        # запоминаем позицию, на которой достигается минимум
        min_x = ninety_degrees[-1]

        for pos in ninety_degrees:
            if (pos not in pluses and \
                (x[pos.i][pos.j] < x[min_x.i][min_x.j])):
                    min_x.i, min_x.j = pos.i, pos.j

        theta = x[min_x.i][min_x.j]

        for pos in ninety_degrees:
            x[pos.i][pos.j] += theta * (1 if pos in pluses else -1)

        # получаем новый базисный план перевозок
        B.remove((min_x.i, min_x.j))

In [4]:
for *data, res in test_cases:
        matrix_transport_problem(*data)
        print(f'\nПравильный ответ:\n{np.array(res, dtype=float)}\n\n')

Базисный план перевозок с минимальной стоимостью:
[[40.  0. 10.]
 [ 0.  0. 50.]
 [ 0. 90. 10.]]

Правильный ответ:
[[40.  0. 10.]
 [ 0.  0. 50.]
 [ 0. 90. 10.]]


Базисный план перевозок с минимальной стоимостью:
[[ 50.   0.  40.]
 [  0.   0.  90.]
 [  0. 100.   0.]]

Правильный ответ:
[[ 50.   0.  40.]
 [  0.   0.  90.]
 [  0. 100.   0.]]


Базисный план перевозок с минимальной стоимостью:
[[100.  40.   0.]
 [  0.   0.  80.]
 [  0.  60.  20.]]

Правильный ответ:
[[100.  40.   0.]
 [  0.   0.  80.]
 [  0.  60.  20.]]


